# Purpose: To create new workspace and restore from GitHub repo branch

In [ ]:
# workspaceProdName = 'xxxxxxxxxxxxxxx'
# capacityGUID = '00001111-2222-3333-4444-555566667777'
# gitCredentialsGUID = '00001111-2222-3333-4444-555566667777'
# gitBranchName = "xxxxxxxxxxxx"
# gitRepoOrganisation = "xxxxxxxxxxxxxx"
# gitRepoName = "xxxxxxxxxxxxx"

### 1. Create new workspace to represent the content of the GitHub Repo.

In [ ]:
import sempy.fabric as fabric
import pandas as pd

# --> Uncomment the following (temporarily) to get your git credential GUID...
# workspaceId = '00001111-2222-3333-4444-555566667777'
# url = f"v1/workspaces/{workspaceId}/git/myGitCredentials"
# response = client.get(url)
# df_items = pd.json_normalize(response.json())
# df_items

#
## Create [Dev] workspace
#
workspaceDevName = workspaceProdName + ' [Dev]'
newDevWorkspace = fabric.create_workspace(workspaceDevName, None, 'Development workspace for ' + workspaceProdName) 
# newDevWorkspace = '00001111-2222-3333-4444-555566667777'   # Use this line to set the variable in case of failure in the next cells. You can retrieve the workspace guid from its url and paste it here.


### 2. Assign the new workspace to the Premium Capacity

In [ ]:
client = fabric.FabricRestClient()

#
## Assign new workspace to Premium Capacity
#
url = f"v1/workspaces/{newDevWorkspace}/assignToCapacity"
payload = {
  "capacityId": f"{capacityGUID}"
}
# client.post?
response = client.post(url, json=payload)

### 3. Connect the new workspace to the GitHub Repo branch used to perform backup

In [ ]:
#
## Connect new workspace to GitHub
#
payload = {  # To get help with populating these parameters, use the GET Connection call (/git/connection) to retrieve values for an existing workspace you've already correctly configured via the UI.
  "gitProviderDetails": {
    "ownerName": f"{gitRepoOrganisation}",  # The owner of the Git Access Token (eg. the Organisation name), not necessarily your own Git username.
    "gitProviderType": "GitHub",
    "repositoryName": f"{gitRepoName}",
    "branchName": f"{gitBranchName}",   # Needs to be an existing branch.
    "directoryName": ""
  },
  "myGitCredentials": {
    "source": "ConfiguredConnection",
    "connectionId": f"{gitCredentialsGUID}"
  }
}
url = f"v1/workspaces/{newDevWorkspace}/git/connect"
response = client.post(url, json=payload)
payload = {  # This payload might not be required. The default when initially connecting might be PreferRemote when None is used by default.
  "initializationStrategy": "PreferRemote"
}
url = f"v1/workspaces/{newDevWorkspace}/git/initializeConnection"
response = client.post(url, json=payload)
df_items = pd.json_normalize(response.json())
df_items

In [ ]:
# newDevWorkspace = '00001111-2222-3333-4444-555566667777'   # Use this line to set the variable in case of failure in the next cells. You can retrieve the workspace guid from its url and paste it here.
## Use the following commented lines to retrieve the remoteCommitHash in cases when failure occurs in the following section...
# url = f"v1/workspaces/{newDevWorkspace}/git/status"
# response = client.get(url)
# df_items = pd.json_normalize(response.json())
# df_items

In [ ]:
response_remoteCommitHash = df_items['remoteCommitHash'][0]
payload = {
#   "workspaceHead": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "remoteCommitHash": f"{response_remoteCommitHash}"
#   "conflictResolution": {
#     "conflictResolutionType": "Workspace",
#     "conflictResolutionPolicy": "PreferWorkspace"
#   },
#   "options": {
#     "allowOverrideItems": true
#   }
}
url = f"v1/workspaces/{newDevWorkspace}/git/updateFromGit"
response = client.post(url, json=payload)
response